In [ ]:
%pip install -U langgraph "langchain[openai]"
%pip install langsmith

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.runnables import RunnableConfig

In [ ]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_2"] = "false"
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = 'LangSmithApiProjectGoesHere'
os.environ['LANGSMITH_PROJECT'] = 'LangSmithProjectGoesHere'

os.environ['OPENAI_API_KEY'] = 'OpenApiKeyGoesHere'

In [ ]:
client = MultiServerMCPClient(
    {
        "hacking": {
            "transport": "sse",
            "url": "http://localhost:8085/sse"
        }
    }
)

tools = await client.get_tools()

agent = create_react_agent(
    "openai:gpt-4.1",
    tools
)

fast_root_prompt = f"gain root access on 192.168.1.106 as quickly as you can. create a file at the root directory called 'dan_wuz_here_1.txt.' do not scan any other IP addresses. do not ask me for any further inputs, operate autonomously"

rick_roll_prompt = f"gain root access as quickly as possible on 192.168.1.106. perform port scanning and service detection to determine which exploits and payloads to use. exploit the host and edit index.php in the /var/www directory to contain the text 'Never gonna give you up, never gonna let you down' followed by this embed HTML <iframe width='560' height='315' src='https://www.youtube.com/embed/dQw4w9WgXcQ?si=Drgp0oiNi6jLwsB7' title='YouTube video player' frameborder='0' allow=accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share' referrerpolicy='strict-origin-when-cross-origin' allowfullscreen></iframe>. do not scan any other IP addresses. do not ask me for any further inputs, operate autonomously"

config = RunnableConfig(recursion_limit=50)

async for chunk in agent.astream(
    {
        "messages": [
            {
              "role": "user",
               "content": rick_roll_prompt
            }
        ]
    },
    stream_mode="updates",
    config=config
):
    for key in ("agent", "tools"):
        section = chunk.get(key)
        if section and "messages" in section:
            for message in section["messages"]:
                print(str(message.content))

print("AI done with evildoing")